<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Before-you-start" data-toc-modified-id="Before-you-start-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Before you start</a></span></li><li><span><a href="#Authentication-setup" data-toc-modified-id="Authentication-setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Authentication setup</a></span></li><li><span><a href="#Hands-off-workflow" data-toc-modified-id="Hands-off-workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hands-off workflow</a></span></li></ul></div>

# Access Sentinel-6 NRT Data

This notebook shows a simple way to maintain a local time series of [Sentinel-6](#) NRT data using the [CMR Search API](#). It downloads granules the ingested since the previous run to a designated data folder and overwrites a hidden file inside with the timestamp of the CMR Search request on success.

> User note: The notebook actually points to a MODIS SST collection for now ([10.5067/GHMDA-2PJ19](https://doi.org/10.5067/GHMDA-2PJ19). It'll work just the same for Sentinel-6.

## Before you start

Before you beginning this tutorial, make sure you have an Earthdata account [https://uat.urs.earthdata.nasa.gov](https://uat.urs.earthdata.nasa.gov).

Accounts are free to create and take just a moment to set up.

## Authentication setup

*You'll probably need to use the netrc method when running from command line.* 

We need some boilerplate up front to log in to Earthdata Login.  The function below will allow Python
scripts to log into any Earthdata Login application programmatically.  To avoid being prompted for
credentials every time you run and also allow clients such as curl to log in, you can add the following
to a `.netrc` (`_netrc` on Windows) file in your home directory:

```
machine uat.urs.earthdata.nasa.gov
    login <your username>
    password <your password>
```

Make sure that this file is only readable by the current user or you will receive an error stating
"netrc access too permissive."

`$ chmod 0600 ~/.netrc` 


In [1]:
from urllib import request
from http.cookiejar import CookieJar
import getpass
import netrc

def setup_earthdata_login_auth(endpoint):
    """
    Set up the request library so that it authenticates against the given Earthdata Login
    endpoint and is able to track cookies between requests.  This looks in the .netrc file 
    first and if no credentials are found, it prompts for them.

    Valid endpoints include:
        uat.urs.earthdata.nasa.gov - Earthdata Login UAT (Harmony's current default)
        urs.earthdata.nasa.gov - Earthdata Login production
    """
    try:
        username, _, password = netrc.netrc().authenticators(endpoint)
    except (FileNotFoundError, TypeError):
        # FileNotFound = There's no .netrc file
        # TypeError = The endpoint isn't in the netrc file, causing the above to try unpacking None
        print('Please provide your Earthdata Login credentials to allow data access')
        print('Your credentials will only be passed to %s and will not be exposed in Jupyter' % (endpoint))
        username = input('Username:')
        password = getpass.getpass()

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, endpoint, username, password)
    auth = request.HTTPBasicAuthHandler(manager)

    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

In [2]:
setup_earthdata_login_auth('uat.urs.earthdata.nasa.gov')

## Hands-off workflow

This workflow/notebook can be run routinely to maintain a time series of NRT data, downloading new granules as they become available in CMR. There are at least a few ways to run it:

* from the notebook server, 
* from the command line with papermill, and
* using nbconvert/papermill with a job scheduler like cron.

The notebook writes a file `.update` to the target data directory the first time it's run. Every subsequent run that finishes successfully overwrites the file with a new timestamp corresponding to the time used for the  granules search parameter `created_at`.

The variables in the cell below determine the workflow behavior in it's initial run. (*`mins` only applies to the first run.*)

In [3]:
#
# This cell accepts parameters from command line with papermill: 
#  https://papermill.readthedocs.io
#

mins = 20  # Limit the results to granules ingested in the last ___ minutes.

cmr = "cmr.uat.earthdata.nasa.gov"  # The domain for CMR or CMR UAT

ccid = "C1234724470-POCLOUD"  # The 'concept-id' of the desired CMR collection

data = "resources/nrt"  # The target path for the NRT granules to download

The variable `data` is pointed at a nearby folder [`resources/nrt`](resources/nrt/) by default. **You should change `data` to a suitable download path on your file system.** An unlucky sequence of git commands could disappear that folder and its downloads, if your not careful. Just change it.

In [4]:
from os import makedirs
from os.path import isdir, basename
from urllib.parse import urlencode
from urllib.request import urlopen, urlretrieve
from datetime import datetime, timedelta
from json import dumps, loads

**The search retrieves granules ingested during the last `n` minutes.** A file in your local data dir  file that tracks updates to your data directory, if one file exists. The CMR Search falls back on the ten minute window if not.

In [5]:
timestamp = (datetime.utcnow()-timedelta(minutes=mins)).strftime("%Y-%m-%dT%H:%M:%SZ")
timestamp

'2020-08-04T23:32:21Z'

This cell will replace the timestamp above with the one read from the `.update` file in the data directory, if it exists.

In [6]:
if not isdir(data):
    print(f"NOTE: Making new data directory at '{data}'. (This is the first run.)")
    makedirs(data)
else:
    try:
        with open(f"{data}/.update", "r") as f:
            timestamp = f.read()
    except FileNotFoundError:
        print("WARN: No .update in the data directory. (Is this the first run?)")
    else:
        print(f"NOTE: .update found in the data directory. (The last run was at {timestamp}.)")

NOTE: .update found in the data directory. (The last run was at 2020-07-28T18:08:02Z.)


There are several ways to query for CMR updates that occured during a given timeframe. Read on in the CMR Search documentation:

* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-new-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#c-with-revised-granules (Collections)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-production-date (Granules)
* https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#g-created-at (Granules)

The `created_at` parameter works for our purposes. It's a granule search parameter that returns the records ingested since the input timestamp.

In [7]:
params = {
    'scroll': "true",
    'page_size': 2000,
    'sort_key': "-start_date",
    'collection_concept_id': ccid, 
    'created_at': timestamp,
    # Limit results to coverage for .5deg bbox in Gulf of Alaska:
    'bounding_box': "-146.5,57.5,-146,58",
}

params

{'scroll': 'true',
 'page_size': 2000,
 'sort_key': '-start_date',
 'collection_concept_id': 'C1234724470-POCLOUD',
 'created_at': '2020-07-28T18:08:02Z',
 'bounding_box': '-146.5,57.5,-146,58'}

Get the query parameters as a string and then the complete search url:

In [8]:
query = urlencode(params)
url = f"https://{cmr}/search/granules.umm_json?{query}"
print(url)

https://cmr.uat.earthdata.nasa.gov/search/granules.umm_json?scroll=true&page_size=2000&sort_key=-start_date&collection_concept_id=C1234724470-POCLOUD&created_at=2020-07-28T18%3A08%3A02Z&bounding_box=-146.5%2C57.5%2C-146%2C58


Get a new timestamp that represents the UTC time of the search. Then request the JSON results and download the records, which should match our search parameters.

In [9]:
timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

with urlopen(url) as f:
    results = loads(f.read().decode())

print(f"{results['hits']} new granules ingested for '{ccid}' since '{timestamp}'.")

462 new granules ingested for 'C1234724470-POCLOUD' since '2020-08-04T23:52:21Z'.


Neatly print the first granule's record for reference (assuming at least one was returned).

In [10]:
if len(results['items'])>0:
    print(dumps(results['items'][0], indent=2))

{
  "meta": {
    "concept-type": "granule",
    "concept-id": "G1236784290-POCLOUD",
    "revision-id": 1,
    "native-id": "20200804194000-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0",
    "provider-id": "POCLOUD",
    "format": "application/vnd.nasa.cmr.umm+json",
    "revision-date": "2020-08-04T23:27:03Z"
  },
  "umm": {
    "RelatedUrls": [
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804194000-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc",
        "Type": "GET DATA",
        "Description": "The base directory location for the granule."
      },
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-public/MODIS_A-JPL-L2P-v2019.0/20200804194000-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.cmr.json",
        "Type": "EXTENDED METADATA",
        "Description": "File to download"
      },
      {
        "URL": "https://archive.podaac.uat.earthdata.nasa.gov/s3c

The link for http access denoted by `"Type": "GET DATA"` in the list of `RelatedUrls`.

Grab the download URL, but do it in a way that'll work for search results returning any number of granule records:

In [11]:
downloads = [r['umm']['RelatedUrls'][0]['URL'] for r in results['items']]
downloads

['https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804194000-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804145501-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804145501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804132001-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804131501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc',
 'https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804131501-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc'

Finish by downloading the files to the data directory in a loop. Overwrite `.update` with a new timestamp on success.

In [12]:
for f in downloads:
    try:
        urlretrieve(f, f"{data}/{basename(f)}")
    except Exception as e:
        print(f"[{datetime.now()}] FAILURE: {f}\n\n{e}\n")
        raise e
    else:
        print(f"[{datetime.now()}] SUCCESS: {f}")

[2020-08-04 19:52:35.286954] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804194000-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:52:42.823872] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804145501-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 19:52:48.547806] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804145501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:52:55.505450] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804132001-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 19:53:01.725230] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200804131501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

[2020-08-04 19:56:36.917772] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200730112001-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:56:42.566667] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200729233501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:56:48.616625] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200729233001-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:56:54.267103] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200729215501-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 19:56:59.603078] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20200729135500-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

[2020-08-04 20:00:49.521612] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20140104214508-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:00:55.130703] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20140104214001-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:01:00.725726] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20140104200508-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:01:05.780750] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20140104200508-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:01:12.598284] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20140104134009-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.

[2020-08-04 20:05:02.501118] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20120610203508-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:05:07.842236] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20120610123507-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:05:13.531215] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20120610123507-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:05:19.720265] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20120610105507-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:05:24.438342] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20120610105507-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

[2020-08-04 20:08:46.960412] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20110728122007-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:08:51.906161] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20110728122007-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:08:58.257389] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20110728104007-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:09:03.087741] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20110728104007-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:09:08.233701] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20110715223008-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

...

[2020-08-04 20:20:00.039116] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090329231008-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:20:05.438424] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090329213508-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:20:11.631388] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090329133508-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:20:18.438171] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090329120008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:20:24.413418] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090329115508-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.

[2020-08-04 20:24:11.919575] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090309122008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:24:17.565598] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090309104008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:24:23.008409] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090308225508-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:24:28.275848] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090308211508-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:24:34.342479] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090308132008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.

[2020-08-04 20:28:45.968346] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090204131507-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:28:52.236320] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090204114007-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:28:58.257822] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090204113507-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:29:03.878495] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090203235007-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:29:09.122593] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090203235007-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

[2020-08-04 20:32:38.435803] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090101133008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:32:44.398764] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090101132508-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:32:50.165426] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20090101115008-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.0.nc
[2020-08-04 20:32:55.385569] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20070912233507-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:33:02.204693] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20070912233007-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.

[2020-08-04 20:36:21.848492] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20040428233006-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:36:27.095949] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20040428215006-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:36:32.458917] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20040428201505-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:36:38.627355] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20040428135006-JPL-L2P_GHRSST-SSTskin-MODIS_A-D-v02.0-fv01.0.nc
[2020-08-04 20:36:44.030129] SUCCESS: https://archive.podaac.uat.earthdata.nasa.gov/podaac-uat-cumulus-protected/MODIS_A-JPL-L2P-v2019.0/20040428135006-JPL-L2P_GHRSST-SSTskin-MODIS_A-N-v02.0-fv01.

If there were updates to the local time series during this run and no exceptions were raised during the download loop, then overwrite the timestamp file that tracks updates to the data folder (`resources/nrt/.update`):

In [13]:
if len(results['items'])>0:
    with open(f"{data}/.update", "w") as f:
        f.write(timestamp)